In [1]:
import numpy as np
from mpi4py import MPI
from dolfinx import log
from dolfinx.io import XDMFFile
from  useful_functions import *

from dolfinx.io.gmshio import read_from_msh

#log.set_log_level(log.LogLevel.WARNING)

# IMPORTING THE MESH #
Vado ad importare la mesh da un file .xdfm oppure da un file .msh
(dovrebbe essere praticamente la stessa cosa)

In [ ]:
from  useful_functions import *

# Potrei leggere la mesh da un file.xdmf, nel seguente modo:
# path_xdmf= "../mesh_files/gianlu_mesh/gian_mesh.xdmf"
# with XDMFFile(MPI.COMM_WORLD, "../mesh_files/gianlu_mesh/gian_mesh.xdmf", "r") as xdmf:
#     domain = xdmf.read_mesh(name='mesh')
#    ct = xdmf.read_meshtags(domain,name='mesh')  #cosa fare con questa riga??


# Oppure potrei leggere la mesh (ed altre info utili) da un file.msh 
path_msh="../mesh_files/michele_mesh/Mesh.msh"
domain, cell_tags, facet_tags=read_from_msh(path_msh,MPI.COMM_WORLD,gdim=2)  


L, H, c_x, c_y, r= L_H_cx_cy_r(domain, facet_tags) #individuo dimensioni del dominio 
# plt_mesh(domain,save_as_png=True) #Plotto o salvo in un png la mesh 

Info    : Reading '../mesh_files/michele_mesh/Mesh.msh'...
Info    : 11733 nodes
Info    : 23466 elements
Info    : Done reading '../mesh_files/michele_mesh/Mesh.msh'


In [3]:
print('H: ',H)
print('L: ',L)
print('')
print(f"Centro ostacolo: ({c_x:.1f}, {c_y:.1f})")
print(f'Raggio: {r:.1f}')

H:  10.0
L:  27.0

Centro ostacolo: (9.0, 5.0)
Raggio: 0.5


# DEFINING THE VECTOR SPACES
#definisco spazi vettoriali di velocità e pressione (Seguendo la legge di Taylor-Hood)

In [4]:
from basix.ufl import element
from dolfinx import fem
v_cg2=element("Lagrange", domain.topology.cell_name(), 2, shape=(domain.geometry.dim,)) # u sarà quadratica 
s_cg1 = element("Lagrange", domain.topology.cell_name(), 1) # p sarà lineare
V = fem.functionspace(domain, v_cg2)
Q = fem.functionspace(domain, s_cg1)

# DEFINING THE BOUNDARIES 

In [5]:
from ufl import SpatialCoordinate
x = SpatialCoordinate(domain)

boundaries = [(1, lambda x: np.isclose(x[0], 0)), # inflow
              (2, lambda x: np.isclose(x[0], L)), # outflow
              (3, lambda x: np.isclose(x[1], 0)), # bottom boundary
              (4, lambda x: np.isclose(x[1], H)), # top boundary 
              (5, lambda x: np.isclose(np.sqrt((x[0] - c_x)**2 + (x[1] - c_y)**2),r))] # disk boundary 

# boundaries è quindi una lista di tuple 
# lambda x: np.isclose(....) saranno i "locator" 
# 1,2,3,4,5 saranno i "marker" 

In [6]:
from dolfinx.mesh import locate_entities, meshtags

facet_indices, facet_markers = [], []
fdim = domain.topology.dim - 1
for (marker, locator) in boundaries:
    facets = locate_entities(domain, fdim, locator) # individuo gli indici delle entities (facets) appartenenti ad un particolare boundary 
    facet_indices.append(facets) # aggiungo gli indici appena trovati a "facet_indices"
    facet_markers.append(np.full_like(facets, marker)) # appendo a facet_marker un array delle stesse dimensioni di quello che conteneva gli 
    # indici appena trovati (ovvero facets) 
    # ma con tutti gli elementi al suo interno uguali al "marker" di riferimento!!! (Quindi, ad esempio, un array di tutti 1)

facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)

sorted_facets = np.argsort(facet_indices) # It returns an array of indices of the same shape as 'facet_indices' that contiene gli indici 
# degli elementi di 'facet_indices' ordinati in maniera crescente 
# Ad esempio, se: facet_indices = np.array([3, 1, 2]) allora: np.argsort(facet_indices) = array([1, 2, 0])


facet_tag = meshtags(domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets]) # Create a MeshTags object 
# facet_indices[sorted_facets] e facet_markers[sorted_facets] saranno 2 array contenenti rispettivamente gli indici degli entities 
# appartenenti ai boundaries (ordinati in maniera crescente)
# ed i rispettivi markers identificativi(1,2,3,4)

Vado a verificare le BC con Paraview

In [7]:
domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)
with XDMFFile(domain.comm, "facet_tags_V2.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_meshtags(facet_tag, domain.geometry)

# DEFINING BC AND VARIATIONAL PROBLEM

In [8]:
from ufl import  TestFunction, TrialFunction

# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

p = TrialFunction(Q)
q = TestFunction(Q)

from dolfinx.fem import Function
# Create useful functions
u0 = Function(V) # ha valore 0 su tutto il dominio (serve per inizializzare u allo step zero)
u1 = Function(V)

p0=Function(Q)
p1 = Function(Q)

In [9]:
#definisco le funzioni che mi serviranno durante il calcolo di mean flow e forzaggio 
u_mean_old = Function(V)
u_mean = Function(V)
u_fluct = Function(V)

f_old = Function(V)
f= Function(V)
f_save = Function(V)

p_mean = Function(Q)

In [10]:
from petsc4py import PETSc

#Defining the constant of the problem 
Re=60 # oppure provare Re=150?
nu=1/Re 
dt=0.01

t=0 # = dt ??
num_steps=600  # 10000
step_init = 10000
step = 1
step_final = 1000000
# t=0.01
# T=...
# n_steps= ...

k=fem.Constant(domain,PETSc.ScalarType(dt)) # time-step 
f_zero= fem.Constant(domain, PETSc.ScalarType((0, 0))) # this should be used only at the very first step!!!

In [11]:
from ufl import FacetNormal,dot, grad, inner 

n = FacetNormal(domain)
# g_u = -(nu*dot(n, grad(u)) - p1*n)
# g_u = -nu* dot(n, grad(u))

g_p= p*n[0]
# g_p= p

In [12]:
from ufl import Measure
ds = Measure("ds", domain=domain, subdomain_data=facet_tag)

Creo una classe che mi aiuterà a definire le BC

In [13]:
from dolfinx.fem import Function, dirichletbc, locate_dofs_topological
from ufl import div, dx, inner, lhs, rhs
from dolfinx import default_scalar_type

In [14]:
gdim=2
#Defining BC 
class InletVelocity():
    def __init__(self, t):
        self.t = t
    def __call__(self, x):
        values = np.zeros((gdim, x.shape[1]), dtype=PETSc.ScalarType)
        values[0] = 4 * 1.5 * np.sin(self.t * np.pi / 8) * x[1] * (0.41 - x[1]) / (0.41**2)
        return values
    
u_inlet_free_slip = Function(V)
inlet_velocity = InletVelocity(t)
u_inlet_free_slip.interpolate(inlet_velocity)
bcu_inflow = dirichletbc(u_inlet_free_slip, locate_dofs_topological(V, fdim, facet_tag.find(1)))

In [15]:
class BoundaryCondition():
    def __init__(self, type, marker, values):
        self._type = type
        if type == "Dirichlet":
            facets = facet_tag.find(marker)
            dofs = locate_dofs_topological(V, fdim, facets)
            self._bc = dirichletbc(values, dofs, V)
        elif type == "Dirichlet_u_y":
            facets = facet_tag.find(marker)
            dofs = locate_dofs_topological(V, fdim, facets)
            self._bc = dirichletbc(values, dofs, V.sub(1))
        elif type == "Dirichlet_p":
            facets = facet_tag.find(marker)
            dofs = locate_dofs_topological(Q, fdim, facets)
            self._bc = dirichletbc(values, dofs, Q)
        elif type == "Neumann":
                self._bc = inner(values, v) * ds(marker)
        elif type == "Neumann_p":
                self._bc = values * q * ds(marker)
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(type))
    @property
    def bc(self):
        return self._bc
    @property
    def type(self):
        return self._type

#Dirichelet values 
u_inlet=PETSc.ScalarType((1,0)) 
u_y_top_bottom=PETSc.ScalarType(0) 
u_disk=PETSc.ScalarType((0,0))
u_no_slip=PETSc.ScalarType((0,0))
p_in=PETSc.ScalarType(8) 
p_out=PETSc.ScalarType(0) 

#Define the BC for step 1
boundary_conditions_step1 = [BoundaryCondition("Dirichlet", 3, u_no_slip), 
                            BoundaryCondition("Dirichlet", 4, u_no_slip), 
                            BoundaryCondition("Dirichlet", 5, u_no_slip)]


#Define the BC for step 2
# boundary_conditions_step2 = [BoundaryCondition("Neumann_p", 2, g_p)]

boundary_conditions_step2 = [BoundaryCondition("Dirichlet_p", 1, p_in),
                             BoundaryCondition("Dirichlet_p", 2, p_out)]

Let's define the 3 Linear problems!!!

In [16]:
#Step 1
#calculation of the tentative velocity u* (by using p(n-1) and u(n-1))
from ufl import inner, nabla_grad, dot, grad, ds, dx, lhs, rhs, div 
F1=(1/k)*inner(u-u0,v)*dx 
# F1+=inner(dot(u0,nabla_grad(u0)),v)*dx 
F1+=inner(grad(u0)*u0,v)*dx
F1+=nu*inner(grad(u),grad(v))*dx
F1-=inner(f_zero,v)*dx

bcu_1, bcp_1 = [],[]
for condition in boundary_conditions_step1:
    if condition.type == "Dirichlet" or condition.type == "Dirichlet_u_y":
        bcu_1.append(condition.bc)
    elif condition.type == "Dirichlet_p":
        bcp_1.append(condition.bc)
    else:
        F1 += condition.bc

############
bcu_1.append(bcu_inflow)
##############
from dolfinx.fem import form 
#definisco quindi la classica forma: a(u,v)=L(v)
a1=form(lhs(F1))
L1=form(rhs(F1))

from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
A1 = assemble_matrix(a1, bcs=bcu_1)
A1.assemble()
b1 = create_vector(L1)

In [17]:
#Step 2 
#caluclation of the new pressure (p_n) (by using u*)
a2=inner(grad(p),grad(q))*dx 

bcu_2, bcp_2 = [],[]
for condition in boundary_conditions_step2:
    if condition.type == "Dirichlet" or condition.type == "Dirichlet_u_y":
        bcu_2.append(condition.bc)
    elif condition.type == "Dirichlet_p":
        bcp_2.append(condition.bc)
    else:
        a2 += condition.bc

a2=form(a2)

L2=-(1/k)*div(u1)*q*dx #non si usa inner o dot (poiché sia div(u1) che q sono degli scalari!)
L2=form(L2)

A2 = assemble_matrix(a2, bcs=bcp_2)
A2.assemble()
b2 = create_vector(L2)

In [18]:
#Step 3 
#calculation of the new velocity (u_n) (by using u* and p_n)
a3=inner(u,v)*dx
a3=form(a3)
L3=inner(u1,v)*dx - k*inner(grad(p1),v)*dx
L3=form(L3)

A3= assemble_matrix(a3, bcs=bcu_1) 
# Ad A3 (e neanche a b3) non c'é bisogno di applicare le BC poiché queste erano già state applicate nei primi 2 step 
# quindi, implicitamente verranno rispettate anche dal risultato dello step 3  
A3.assemble()
b3= create_vector(L3)

Adesso che quindi abbiamo definito tutte le strutture necessarie per i problemi lineare possiamo creare quindi i vari 
solver utilizzando PETSc (possiamo anche customizzare la strategia di risoluzione pre ogni problema)

For the tentative velocity step and pressure correction step, we will use the Stabilized version of BiConjugate Gradient to solve the linear system, and using algebraic multigrid for preconditioning. 

For the last step, the velocity update, we use a conjugate gradient method with successive over relaxation, Gauss Seidel (SOR) preconditioning.

In [19]:
# Solver for step 1
solver1 = PETSc.KSP().create(domain.comm)
solver1.setOperators(A1)
solver1.setType(PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(PETSc.PC.Type.HYPRE)
pc1.setHYPREType("boomeramg")

# Solver for step 2
solver2 = PETSc.KSP().create(domain.comm)
solver2.setOperators(A2)
solver2.setType(PETSc.KSP.Type.BCGS)
pc2 = solver2.getPC()
pc2.setType(PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver for step 3
solver3 = PETSc.KSP().create(domain.comm)
solver3.setOperators(A3)
solver3.setType(PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(PETSc.PC.Type.SOR)

# SOLVING THE VARIATIONAL PROBLEM

In [20]:
from pathlib import Path
from dolfinx.io import VTXWriter
from tqdm.notebook import tqdm
#from tqdm.autonotebook import tqdm


folder = Path("results_V2")
folder.mkdir(exist_ok=True, parents=True)

vtx_u = VTXWriter(domain.comm, folder/"flow_u.bp", [u0], engine="BP4") 
vtx_p = VTXWriter(domain.comm, folder/"pressure_p.bp", [p1], engine="BP4") 

#associo il tempo t 
vtx_u.write(t)
vtx_p.write(t)

#progress = tqdm.autonotebook.tqdm(desc="Solving PDE", total=num_steps)
progress = tqdm(desc="Solving PDE", total=num_steps)    #serve solo per visualizzare il progresso (in %) della solzuione

Solving PDE:   0%|          | 0/600 [00:00<?, ?it/s]

Quando scrivi un file .bp con engine="BP4" (come in VTXWriter), non viene creato un singolo file leggibile come testo, 

ma un container binario suddiviso in più file "fisici". 

Questo è normale e previsto dal funzionamento interno di ADIOS2.

Quindi, ad esempio, all’interno della directory "poiseuille_u.bp", troverò:

-data.0    <---Contiene i dati numerici binari grezzi (cioè valori dei campi FEM, vettori, ecc.).

-md.0    <---Contiene i metadati (descrizione del contenuto, nomi delle variabili, topologia).

-md.idx  <---È un indice rapido dei metadati, usato per l’accesso efficiente e la parallelizzazione.

Tutti insieme, questi 3 file compongono il file .bp, e devono essere trattati come un tutt'uno.



In ParaView, puoi aprire direttamente poiseuille_u.bp e ParaView si occuperà di interpretare correttamente data.0, md.0, e md.idx.

In [21]:
from dolfinx.fem import apply_lifting
from dolfinx.fem.petsc import LinearProblem

for i in range(num_steps):  #al posto di num_steps (in realtà dovrebbe esserci step_final!!!!!!)
    progress.update(1)
    # Update current time step
    t += dt


    # Step 1: Tentative velocity step
    with b1.localForm() as loc_1:
        loc_1.set(0)
    assemble_vector(b1, L1)
    apply_lifting(b1, [a1], [bcu_1])
    b1.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b1, bcu_1)
    solver1.solve(b1, u1.x.petsc_vec)
    u1.x.scatter_forward()

    # Step 2: Pressure corrrection step
    with b2.localForm() as loc_2:
        loc_2.set(0)
    assemble_vector(b2, L2)
    apply_lifting(b2, [a2], [bcp_2])
    b2.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b2, bcp_2)
    solver2.solve(b2, p1.x.petsc_vec)
    p1.x.scatter_forward()

    # Step 3: Velocity correction step
    with b3.localForm() as loc_3:
        loc_3.set(0)
    assemble_vector(b3, L3)
    apply_lifting(b3, [a3], [bcu_1])
    b3.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b3, bcu_1)
    solver3.solve(b3, u1.x.petsc_vec)
    u1.x.scatter_forward()

    if i > step_init:
        actual_step=i-step_init
        
        u_mean.x.array[:]=(actual_step/(actual_step+1))*u_mean.x.array[:] + 1/(actual_step+1)*u1.x.array[:]
        p_mean.x.array[:]=(actual_step/(actual_step+1))*p_mean.x.array[:] + 1/(actual_step+1)*p1.x.array[:]

        u_fluct.x.array[:]=u1.x.array[:]-u_mean.x.array[:]

        ####### Calcolo di f########
        # voglio calcolare f, che risolta uguale a: f_prod= -dot(grad(u_fluct), u_fluct)
        expr = dot(grad(u_fluct), u_fluct)
        # Problema variazionale per la proiezione in V
        u_f = fem.TrialFunction(V)
        v_f = fem.TestFunction(V)
        a_f = inner(u_f, v_f) * dx
        L_f = inner(expr, v_f) * dx
        # Risolvi la proiezione L2
        problem = LinearProblem(a_f, L_f, bcs=[], petsc_options={"ksp_type": "cg"})
        f_prod = -problem.solve()  # prod è una Function nello spazio V
        
        f.x.array[:]=(actual_step/(actual_step+1))*f_old.x.array[:] + 1/(actual_step+1)*f_prod.x.array[:]

        #calcolo la differenza (norma) fra le variabili calcolate (mean_flow e forcing) in 2 iterazioni successive
        # mi serve per poter verificare la convergenza  
        err_mean = np.linalg.norm((u_mean_old.vector()[:] - u_mean.vector()[:])) / np.linalg.norm(u_mean_old.vector()[:])
        err_forc = np.linalg.norm((f_old.vector()[:] - f.vector()[:])) / np.linalg.norm(f_old.vector()[:])




        if (err_mean < 1e-15 and err_forc < 1e-15) or step == step_final:
            # cfl_max, dt_min = CFLnumber(domain, u1, dt)
            '''Dobbiamo andare a calcolare il CFL number!!!!!!
            Ma questa serà una cosa che vedrò una volta risolto il problema del crashing della simulazione!!!!!'''

            MPI.COMM_WORLD.Barrier()

            if step == step_final:
                print(f'Last step reached: L2 mean:{err_mean}, L2 forc:{err_forc}')
                #print(f"Last step reached: L2 mean:{err_mean}, L2 forc:{err_forc}, CFL = {cfl_max}, dt = {dt_min}")
            else:
                print(f'Convergence reached: L2 mean:{err_mean}, L2 forc:{err_forc}')
                #print(f'Convergence reached: L2 mean:{err_mean}, L2 forc:{err_forc}, CFL = {cfl_max}, dt = {dt_min}')

            MPI.COMM_WORLD.Barrier()
            break

        
    if i in range(50,55):
        print(f'STEP {i}')
        print( f'u{i} = ', u1.x.array[:])
        #print(f'Somma su u{i} = ',sum(u1.x.array[:]))
        
        print( f'p{i} = ', p1.x.array[:])
        #print(f'Somma su p{i} = ',sum(p1.x.array[:]))
        
        print('')
        
    # Let's move to the next step!!!!!
    # Update variable with solution form this time step
    u0.x.array[:] = u1.x.array[:] # aggiorno il valore della u0 (che rappresenta u(n-1) nello step 1 dell'IPCS) 
    
    f_old.x.array[:]=f.x.array[:] # aggiono il valore del forzaggio 
    p0.x.array[:] = p1.x.array[:] # aggiorno il valore della p0
    
    # Write solutions to file
    vtx_u.write(t)
    vtx_p.write(t)
    
    # # Compute error at current time-step
    # error_L2 = np.sqrt(mesh.comm.allreduce(assemble_scalar(L2_error), op=MPI.SUM))
    # error_max = mesh.comm.allreduce(np.max(u_.x.petsc_vec.array - u_ex.x.petsc_vec.array), op=MPI.MAX)
    # # Print error only every 20th step and at the last step
    # if (i % 20 == 0) or (i == num_steps - 1):
    #     print(f"Time {t:.2f}, L2-error {error_L2:.2e}, Max error {error_max:.2e}")
# Close xmdf file
vtx_u.close()
vtx_p.close()
b1.destroy()
b2.destroy()
b3.destroy()
solver1.destroy()
solver2.destroy()
solver3.destroy()

STEP 50
u50 =  [ 1.49287784e-01 -3.22049297e-06  1.49301133e-01 ...  1.50640331e-03
  0.00000000e+00  0.00000000e+00]
p50 =  [0.         0.         0.06406442 ... 7.86919587 7.99998699 7.99998699]

STEP 51
u51 =  [ 1.52200239e-01 -3.35181437e-06  1.52214070e-01 ...  1.54320252e-03
  0.00000000e+00  0.00000000e+00]
p51 =  [0.         0.         0.06405254 ... 7.8673387  7.99998699 7.99998699]

STEP 52
u52 =  [ 1.55112126e-01 -3.48560345e-06  1.55126444e-01 ...  1.58003645e-03
  0.00000000e+00  0.00000000e+00]
p52 =  [0.         0.         0.06404062 ... 7.86546709 7.99998698 7.99998698]

STEP 53
u53 =  [ 1.58023445e-01 -3.62184757e-06  1.58038255e-01 ...  1.61689782e-03
  0.00000000e+00  0.00000000e+00]
p53 =  [0.         0.         0.06402866 ... 7.86358095 7.99998698 7.99998698]

STEP 54
u54 =  [ 1.60934193e-01 -3.76053371e-06  1.60949501e-01 ...  1.65377940e-03
  0.00000000e+00  0.00000000e+00]
p54 =  [0.         0.         0.06401666 ... 7.86168019 7.99998698 7.99998698]

